## Importing Libraries

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model # this is to load the model from the file system
import pickle
import pandas as pd

## Loading model , preprocessing and prediction



In [3]:
# Load the model
model = load_model('model.h5')

# load the encoder and scaler
with open('label_encoders_gender.pkl', 'rb') as file:
    label_encoders = pickle.load(file) # we load the label encoders from the file into the label_encoders variable
    
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file) # we load the scaler from the file into the scaler variable
    
with open('onehot_encoders_geo.pkl', 'rb') as file:
    one_hot_encoder = pickle.load(file) # we load the one hot encoder from the file into the one_hot_encoder variable

## Data for the model 

In [4]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age' : 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

## Encode and Scale the data

In [12]:
# One hot encode the geography
geo = one_hot_encoder.transform(np.array(input_data['Geography']).reshape(-1,1)).toarray() # no need to convert to 2d array as we are using numpy array and not pandas dataframe
geo = pd.DataFrame(geo, columns=one_hot_encoder.get_feature_names_out(['Geography']))
geo 

c:\Users\BHAVYASHA\Desktop\languages\Projects\AI\ANN_PRO_1\env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [13]:
# convert the input data into a pandas dataframe
input_data_df = pd.DataFrame([input_data])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [14]:
# Label encode
input_data_df['Gender'] = label_encoders.transform(input_data_df['Gender'])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [15]:
# Concatenate the dataframes of the input data and the one hot encoded geography
input_data_df = pd.concat([input_data_df.drop('Geography',axis=1), geo], axis=1)
input_data_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [16]:
# Scale the input data
input_data_scaled = scaler.transform(input_data_df)
input_data_scaled

array([[-0.52111599,  0.91601335,  0.10961719, -0.68538967, -0.2569057 ,
         0.8095029 ,  0.64259497,  0.9687384 , -0.87203322,  0.98560362,
        -0.5698444 , -0.57369368]])

In [17]:
# Make predictions using the model
predictions = model.predict(input_data_scaled)
predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 659ms/step


array([[0.03515014]], dtype=float32)

In [19]:
if predictions > 0.5:
    print('The customer is likely to leave the bank')
else:
    print('The customer is likely to stay with the bank')

The customer is likely to stay with the bank
